In [1]:
#import statements

from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
np.random.seed(1671) # for reproducibility
import os
import itertools
from itertools import repeat

#model based imports
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.core import Activation
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Input, Conv2D, Dense
from keras.layers import Conv2D
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
#paths to image folders for training
category1_path = "/custom_path/all_images_folder/category1"
category2_path = "/custom_path/all_images_folder/category2"
# ......
category10_path = "/custom_path/all_images_folder/category10"

#Create list of category paths for easier use
training_paths = [category1_path, category2_path, ... , category10_path]

#path to image folders for testing
testing_category1_path = "/custom_path/all_images_folder/test_images/category1"
testing_category2_path = "/custom_path/all_images_folder/test_images/category2"
# ......
testing_category10_path = "/custom_path/all_images_folder/test_images/category10"

##Create list of testing category paths for easier use
testing_paths = [testing_category1_path, testing_category2_path, ... , testing_category10_path]

In [3]:
#Set values

epochs = 100
batch_size = 128
verbose = 1
classes = 10 # number of outputs = number of categories
optimizer = Adam() # use optimizer of choice
val_split=0.2 # how much training data is reserved for validation

#Image dimensions
img_height = 100
img_width = 100

#Make list of number of classes
categories = []
for i in range(classes):
    categories.append(i)
    
#Set limits for each image folder for training data
category1 = 100 #max 200
category2 = 50 #max 100
# ...
category10 = 200 #max 940

#Create a list of limits for easier use
train_limits = [category1, category2, ... , category10]

#Set limits for each image folder for testing data
teat_category1 = 20 #max 50
test_category2 = 20 #max 60
# ...
test_category10 = 20 #max 100

#Create a list of limits for easier use
test_limits = [test_category1, test_category2, ... , test_category10]

In [4]:
# Function that uses os.listdir to load images
def load_images(x, paths, limits):
    cat1_images = os.listdir(paths[0])[:limits[0]]
    cat2_images = os.listdir(paths[1])[:limits[1]]
    cat3_images = os.listdir(paths[2])[:limits[2]]
    # ...
    cat10_images = os.listdir(paths[9])[:limits[9]]
    
    #load images for category 1
    for i in range(limits[0]):
        #load image
        img = load_img(paths[0] + bio_images[i], target_size = (img_height,img_width), color_mode = "grayscale")
        #convert to numpy array
        training_img_array = img_to_array(img, dtype="float32")
        x.append(training_img_array)
    
    #load images for category 2
    for i in range(limits[1]):
        #load image
        img = load_img(paths[1] + fibre_images[i], target_size = (img_height,img_width), color_mode = "grayscale")
        #convert to numpy array
        training_img_array = img_to_array(img, dtype="float32")
        x.append(training_img_array)
    
    # .....
    
    #load images for category 10
    for i in range(limits[9]):
        #load image
        img = load_img(paths[9] + tips_images[i], target_size = (img_height,img_width), color_mode = "grayscale")
        #convert to numpy array
        training_img_array = img_to_array(img, dtype="float32")
        x.append(training_img_array)
    
    

In [5]:
# Function to create labels
def create_labels(y, cat, limits):
    for (i,j) in zip(cat,limits):
        y.extend(repeat(i,j))
    

In [6]:
#Create training dataset and labels
x_train = []
y_train = []

load_images(x_train, training_paths, train_limits)
print("Number of training images used: " + str(len(x_train)))

create_labels(y_train, categories, train_limits)
print("Number of training labels created: " + str(len(y_train)))

#convert x_train from list to nparray of float32
x_train = np.asarray(x_train)
x_train/=255
print("Shape of training data: " + str(x_train.shape) + "Datatype: " + str(x_train.dtype))

#Converting the labels to arrays
y_train = np.asarray(y_train, dtype="uint8")

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, classes)

print("Training labels shape: " + str(Y_train.shape) + "Datatype: " + str(Y_train.dtype))

Number of training images used: 18477
Number of training labels created: 18477
Shape of training data: (18477, 100, 100, 1)Datatype: float32
Training labels shape: (18477, 10)Datatype: float32


In [7]:
#Create testing dataset and labels
x_test = []
y_test = []

load_images(x_test, testing_paths, test_limits)
print("Number of testing images used: " + str(len(x_test)))

create_labels(y_test, categories, test_limits)
print("Number of testing labels created: " + str(len(y_test)))

#convert x_train from list to nparray of float32
x_test = np.asarray(x_test)
x_test/=255
print("Shape of testing data: " + str(x_test.shape) + "Datatype: " + str(x_test.dtype))

#Converting the labels to arrays
y_test = np.asarray(y_test, dtype="uint8")

# convert class vectors to binary class matrices
Y_test = np_utils.to_categorical(y_test, classes)

print("Testing labels shape: " + str(Y_test.shape) + "Datatype: " + str(Y_test.dtype))

Number of testing images used: 100
Number of testing labels created: 100
Shape of testing data: (100, 100, 100, 1)Datatype: float32
Testing labels shape: (100, 10)Datatype: float32


In [8]:
#Model architecture to use inception v3 for grayscale images

inception = InceptionV3(weights='imagenet', include_top = True)
input_tensor = Input(shape=(img_height,img_width,1))
x = Conv2D(3,(3,3),padding='same')(input_tensor)    # x has a dimension of (img_height,img_width,3)
out1 = inception(x)
out = Dense(10, activation = 'softmax')(out1)

model = Model(inputs = input_tensor, outputs = out)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 100, 100, 3)       30        
_________________________________________________________________
inception_v3 (Model)         (None, 1000)              23851784  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10010     
Total params: 23,861,824
Trainable params: 23,827,392
Non-trainable params: 34,432
_________________________________________________________________


In [9]:
#Compile model

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
#Fit model

history = model.fit(x_train, Y_train, batch_size = batch_size, epochs = epochs)

Epoch 1/10
  256/18477 [..............................] - ETA: 13:24:00 - loss: 2.3041 - accuracy: 0.02 - ETA: 17:16:38 - loss: 2.3018 - accuracy: 0.1172

In [ ]:
#Evaluate testing score

score = model.evaluate(x_test, Y_test, verbose = verbose)
print("Test score:", score[0])
print('Test accuracy:', score[1])

In [ ]:
#Calculate predictions

y_pred = model.predict(x_test)
predictions = np.argmax(y_pred, axis = 1)

print(predictions[:20])
labels_pred = np.argmax(Y_test, axis= 1)
print(labels_pred[:20])